# Scarico gli id dei replay

In [13]:
import json
replay = []
i=0
with open('gen5_random_battle_replays.jsonl', 'r') as f:
    for lines in f:
        l = json.loads(lines)
        if l['rating']!= None:
#            if l['rating'] >= 1000:
            replay.append(l['id'].split('-')[-1])

In [14]:
len(replay)

13816

# Scarico i relativi log

In [15]:
import requests
from requests.exceptions import JSONDecodeError
from tqdm import tqdm
import json
import time
import re

output_file = 'pokemon_battles_dataset_1500.jsonl'
batch_size = 100
processed_count = 0
dataset = []

def save_dataset(dataset, mode='a'):
    with open(output_file, mode) as f:
        for item in dataset:
            json.dump(item, f)
            f.write('\n')
    print(f"Saved {len(dataset)} entries to {output_file}")

def clean_battle_log(log):
    # Remove timer warnings, chat messages, and other unnecessary information
    cleaned_log = []
    for line in log.split('\n'):
        if any(x in line for x in ['|t:', '|inactive|', '|player|', '|c|', '|l|', '|j|']):
            continue
        cleaned_log.append(line)
    return '\n'.join(cleaned_log)

try:
    for id in tqdm(replay, desc="Processing battles"):
        try:
            response = requests.get(f'https://replay.pokemonshowdown.com/gen5randombattle-{id}.json')
            if response.status_code != 200:
                tqdm.write(f"Skipping battle {id} due to non-200 status code: {response.status_code}")
                continue
            
            battle = response.json()
        except JSONDecodeError:
            tqdm.write(f"Skipping battle {id} due to invalid JSON response")
            continue
        
        players = battle['players']
        battle = battle['log'].replace(players[0], 'p1a').replace(players[1], 'p2a').split('\n')
        if '|turn|6' not in battle: 
            tqdm.write(f"Skipping battle {id} as it doesn't reach turn 6")
            continue
        battle = battle[battle.index('|start'):] # taglio via i log precedenti a  di |start
        team = {}
        # Estraggo i pokemon e le moves
        for row in battle:
            if '|switch|p1a:' in row:
                pkm = row[13:].strip().split('|')[1]
                key_pkm = row[13:].strip().split('|')[0]
                pkm = pkm.split(', ') if ', ' in pkm else [pkm.strip(), '100']
                if key_pkm not in team.keys():
                    team[key_pkm]={'names': pkm[0], 'liv': pkm[1], 'moves': [], 'abilità': ''}
            if '|move|p1a:' in row:
                line = row[11:].split('|')
                pkm, move = line[0], line[1]
                if team.get(pkm) and move not in team.get(pkm)['moves']:
                    team.get(pkm)['moves'].append(move)

        # Divido in turni:
        turni, actions= {}, {}
        nturno = 0
        for index, row in enumerate(battle):
            if '|turn|' in row:
                nturno = int(row.strip('|turn|').strip())
                current_turn = battle[:index]
                pkm, pkm_avv = False, False
                for row0 in current_turn[::-1]:
                    if '|switch|p1a: ' in row0:
                        pkm = row0[13:].split('|')[0]
                    if '|switch|p2a: ' in row0:
                        pkm_avv = row0[13:].split('|')[0]
                    if pkm and pkm_avv: break
                turni[nturno-1] = {'pkm': pkm, 
                                   'pkm avversario': pkm_avv, 
                                   'morti': [r[12:] for r in current_turn if '|faint|p1a: ' in r],
                                   'log': clean_battle_log('\n'.join(current_turn))}
            if nturno > 0:
                if '|move|p1a: ' in row:
                    move = row[11:].split('|')[1]
                    actions[nturno] = f'move|{move}'
                    continue
                if '|switch|p1a: ' in row:
                    pkm = row[13:].split('|')[0]
                    actions[nturno] = f'switch|{pkm}'
                    continue

        prompt="""You're in a Pokémon battle at turn {}.
        You've this pokémon in the field: {}.
        Your opponent have this pokémon: {}.
        You have these switches available:
        {}
        Read carefully the log and choose the action you want to do:"""
        
        for t in range(1,  nturno-1):
            dataset.append(
                [
                    {'role': 'system',
                     'content': prompt.format(*[
                        t,
                        str(turni[t-1]['pkm']+' '+str(team[turni[t-1]['pkm']])),
                        turni[t-1]['pkm avversario'],
                        '\n'.join([f"- {p} | {info}{' | fainted' if p in turni[t-1]['morti'] else ''}" for p, info in team.items() if p!=turni[t-1]['pkm']])
                        ])
                        },
                    {'role': 'user', 'content': turni[t-1]['log']},
                    {'role': 'assistant', 'content': actions.get(t)}
                ]
            )
        
        processed_count += 1
        if processed_count % batch_size == 0:
            save_dataset(dataset)
            dataset = []  # Clear the dataset after saving
        
        time.sleep(0.1)

except KeyboardInterrupt:
    print("\nScript interrupted. Saving current data...")
    if dataset:
        save_dataset(dataset)
    print("Data saved. Exiting.")

# Save any remaining data if the script completes normally
if dataset:
    save_dataset(dataset)
print("Script completed successfully.")


Processing battles:   0%|          | 1/13816 [00:00<1:15:26,  3.05it/s]

Skipping battle 2149614159 as it doesn't reach turn 6


Processing battles:   0%|          | 2/13816 [00:00<1:20:32,  2.86it/s]

Skipping battle 2149613931 as it doesn't reach turn 6


Processing battles:   0%|          | 17/13816 [00:07<1:31:07,  2.52it/s]

...

Skipping battle 818098 due to non-200 status code: 404


Processing battles: 100%|█████████▉| 13815/13816 [1:57:20<00:00,  1.15it/s]

Skipping battle 624166 due to non-200 status code: 404


Processing battles: 100%|██████████| 13816/13816 [1:57:21<00:00,  1.96it/s]


Skipping battle 543390 due to non-200 status code: 404
Saved 1661 entries to pokemon_battles_dataset_1500.jsonl
Script completed successfully.
